**OBJECTIVE:** Test the process of embedding a sample of PDFs and saving them into a DuckDB database

**AUTHOR:** [Aksh Sabherwal](https://www.github.com/akshsabherwal) (edited by [@jonjoncardoso](https://github.com/jonjoncardoso))

⚙️ **SETUP**

- Ensure you are running with the `chat-lse` conda environment. See [README.md](../../README.md) for more information.
- Install the packages we will need for this experiment:

    ```bash
    pip install PyPDF2 duckdb duckdb-engine sentence-transformers
    ```

**Imports**

In [29]:
import os
import duckdb
import PyPDF2

from typing import List
from pprint import pprint
from pydantic import BaseModel
from tqdm.notebook import tqdm
from fastapi import FastAPI, HTTPException
from sentence_transformers import SentenceTransformer


**Functions**

In [23]:
# Function to insert documents
def insert_document(conn, id, title, content, vec):
    conn.execute('INSERT INTO embeddings (id, title, content, vec) VALUES (?, ?, ?, ?)', (id, title, content, vec))

# Extract text from PDFs using PyPDF2
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in tqdm(reader.pages):
            text += page.extract_text()
    return text

def keyword_search(conn, keyword):
    query = '''
    SELECT id, title, content FROM embeddings
    WHERE title ILIKE ? OR content ILIKE ?
    '''
    return conn.execute(query, [f'%{keyword}%', f'%{keyword}%']).fetchall()

def vector_search(conn, query_vector):
    query_vector_str = ', '.join(map(str, query_vector))
    query = f'''
    SELECT id, title, content, array_distance(vec, ARRAY[{query_vector_str}]::FLOAT[384]) as distance
    FROM embeddings
    ORDER BY distance
    LIMIT 3
    '''
    return conn.execute(query).fetchall()

def unified_search(conn, keyword, query_vector):
    keyword_results = keyword_search(conn, keyword)
    vector_results = vector_search(conn, query_vector)

    # Convert results to a dictionary for easier combination and elimination of duplicates
    results_dict = {result[0]: result for result in keyword_results}
    for result in vector_results:
        if result[0] not in results_dict:
            results_dict[result[0]] = result

    # Combine and sort results (you can choose your own sorting strategy)
    combined_results = list(results_dict.values())
    combined_results.sort(key=lambda x: x[3] if len(x) > 3 else float('inf'))

    return combined_results

# 1. Set up the database

Create the database file or remove it if it already exists.

In [4]:
# Define database file path
db_path = '../../data/mydb.duckdb'

# Delete the WAL file if it exists
wal_path = db_path + '.wal'
if os.path.exists(wal_path):
    os.remove(wal_path)

# Connect to DuckDB (use a file-based database)
conn = duckdb.connect(database=db_path)

# Install and load the VSS extension
conn.execute("INSTALL vss")
conn.execute("LOAD vss")

# Enable experimental persistence for HNSW indexes after loading VSS extension
conn.execute("SET hnsw_enable_experimental_persistence=true")

(Re)-create the table EMBEDDING:

In [5]:
conn.execute('DROP TABLE IF EXISTS embeddings')

conn.execute(
'''
CREATE TABLE embeddings (
    id INTEGER PRIMARY KEY,
    title TEXT,
    content TEXT,
    vec FLOAT[384]
)
'''
)

conn.execute('CREATE INDEX hnsw_index ON embeddings USING HNSW (vec)')

# 2. Extract text from PDFs

I added two sample documents for tests:

In [6]:
exam_procedures_path = '../../data/sample-docs/Exam-Procedures-for-Candidates.pdf'
exam_timetable_path = '../../data/sample-docs/Spring-Exam-Timetable-2024-Final.pdf'

Get the text from the PDFs above using the PyPDF2 library:

In [7]:
exam_procedures_text = extract_text_from_pdf(exam_procedures_path)
exam_timetable_text = extract_text_from_pdf(exam_timetable_path)

incorrect startxref pointer(1)


  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

Check the text extracted:

In [18]:
print(f"First 500 characters from Exam Procedures PDF:\n{'-'*100}\n{exam_procedures_text[:500]}{'-'*100}\n")
print(f"First 500 characters from Exam Timetable PDF:\n{'-'*100}\n{exam_timetable_text[:500]}\n{'-'*100}")

First 500 characters from Exam Procedures PDF:
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------

First 500 characters from Exam Timetable PDF:
----------------------------------------------------------------------------------------------------
Exam Code Exam Name Start Time
AC100 Elements of Accounting and Finance - Spring Resit/Deferred syllabus years pre 2023/24 14: 30
AC102 Elements of Financial Accounting -  Spring Exam and Spring Resit/Deferred syllabus years pre 2023/24 14: 30
AC103Elements of Management Accounting, Financial Management and Financial Institutions -  Spring Exam and 
Spring Resit/Deferred syllabus years pre 2023/2410:00
AC105 Introduction to Financial Accounting - Spring Exam 14: 30
AC106 Introduction to Manageme
----------------------------------------------------------------------------------------------------


<span style="font-weight:bold;color:red;font-size:1.4em">TODO:</span> Check what the error `incorrect startxref pointer(1)` means for the first PDF and why it is not extracting the text.

# 3. Embed the text using a HuggingFace model 

In [20]:
# Convert text to embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# exam_procedures_embedding = model.encode(exam_procedures_text)
exam_timetable_embedding = model.encode(exam_timetable_text)

# Inserting the documents
# insert_document(1, 'Exam Procedures for Candidates', exam_procedures_text, exam_procedures_embedding.tolist())
insert_document(conn, 1, 'Spring Exam Timetable 2024', exam_timetable_text, exam_timetable_embedding.tolist())

c:\Users\Jon\miniconda3\envs\chat-lse\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# 4. Experiment running queries 

In [34]:
# Example usage
keyword = "exam"
query_vector = model.encode("exam procedures").tolist()  # Replace with the actual query vector
results = unified_search(conn, keyword, query_vector)
for result in results:
    pprint(result)

(1,
 'Spring Exam Timetable 2024',
 'Exam Code Exam Name Start Time\n'
 'AC100 Elements of Accounting and Finance - Spring Resit/Deferred syllabus '
 'years pre 2023/24 14: 30\n'
 'AC102 Elements of Financial Accounting -  Spring Exam and Spring '
 'Resit/Deferred syllabus years pre 2023/24 14: 30\n'
 'AC103Elements of Management Accounting, Financial Management and Financial '
 'Institutions -  Spring Exam and \n'
 'Spring Resit/Deferred syllabus years pre 2023/2410:00\n'
 'AC105 Introduction to Financial Accounting - Spring Exam 14: 30\n'
 'AC106 Introduction to Management Accounting - Spring Exam 10: 00\n'
 'AC200 Accounting Theory and Practice -  Spring Exam and Spring '
 'Resit/Deferred syllabus years pre 2023/24 14: 30\n'
 'AC311 Results Accountability and Management Control for Strategy '
 'Implementation -  Spring Exam 10: 00\n'
 'AC331 Contemporary Issues in Financial Accounting -  Spring Resit/Deferred '
 'syllabus years pre 2023/24 10: 00\n'
 'AC332Financial Statement Analys